In [1]:
import numpy as np
import argparse
import csv
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
np.random.seed(42)
import math
import random
random.seed(42)
import pandas as pd

In [4]:
MAX_POWER_M = 3
MAX_POWER_Y = 3
MAX_POWER_MY = 3
N_FOLDS = 10
REGULARIZATION_LAMBDA = 0.05

In [5]:
df = pd.read_csv('train.csv')

In [6]:
df

,id,value
0,11/1/04,0.5992
1,10/1/12,0.9259
2,1/1/07,5.9182
3,2/1/06,9.2365
4,6/1/07,22.7446
...,...,...
105,9/1/06,3.2992
106,4/1/06,22.0183
107,7/1/05,22.2829
108,6/1/08,23.1280


In [7]:
df[['month','date','year']] = df['id'].str.split('/',expand = True)

In [8]:
df

,id,value,month,date,year
0,11/1/04,0.5992,11,1,04
1,10/1/12,0.9259,10,1,12
2,1/1/07,5.9182,1,1,07
3,2/1/06,9.2365,2,1,06
4,6/1/07,22.7446,6,1,07
...,...,...,...,...,...
105,9/1/06,3.2992,9,1,06
106,4/1/06,22.0183,4,1,06
107,7/1/05,22.2829,7,1,05
108,6/1/08,23.1280,6,1,08


In [9]:
df[df['month']=='5']

,id,value,month,date,year
13,5/1/07,23.4412,5,1,07
32,5/1/06,31.6249,5,1,06
38,5/1/09,28.3363,5,1,09
47,5/1/14,24.2971,5,1,14
48,5/1/05,30.1723,5,1,05
64,5/1/11,26.2492,5,1,11
85,5/1/12,23.0200,5,1,12
101,5/1/08,25.1503,5,1,08
104,5/1/13,26.6731,5,1,13


In [10]:
df[df['year']=='05']

,id,value,month,date,year
12,1/1/05,-1.2773,1,1,05
24,11/1/05,-1.0235,11,1,05
44,10/1/05,6.6850,10,1,05
46,2/1/05,8.9098,2,1,05
48,5/1/05,30.1723,5,1,05
53,6/1/05,25.3528,6,1,05
58,12/1/05,-3.8558,12,1,05
63,9/1/05,6.7498,9,1,05
92,8/1/05,12.2146,8,1,05
96,4/1/05,23.9407,4,1,05


In [11]:
df[df['year']=='06']

,id,value,month,date,year
3,2/1/06,9.2365,2,1,06
10,6/1/06,27.1375,6,1,06
32,5/1/06,31.6249,5,1,06
36,7/1/06,23.1523,7,1,06
37,12/1/06,-5.4002,12,1,06
54,11/1/06,1.0582,11,1,06
55,8/1/06,12.3766,8,1,06
60,10/1/06,-1.7417,10,1,06
94,3/1/06,17.0746,3,1,06
105,9/1/06,3.2992,9,1,06


In [12]:
dataset = []
for row in df.iterrows():
    dataset.append([int(row[1]['month']),int(row[1]['year']),int(row[1]['month'])*int(row[1]['year']),float(row[1]['value'])])

In [51]:
def normalize(des):
    

In [13]:
def cross_validation_split(dataset):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / N_FOLDS)
    for i in range(N_FOLDS):
        fold = list()
        while len(fold) < fold_size:
            index = random.randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [26]:
# highest power is m
def makeDesignMat(x1,x2,x3,m,y,my):
    m1 =  np.vander(x1,m+1,increasing=True)
    m2 = np.vander(x2,y+1,increasing=True)
    m3 = np.vander(x3,my+1,increasing=True)
    temp = np.concatenate((m1,m2[:,1:]),axis=1)
    return np.concatenate((temp,m3[:,1:]),axis=1)
#makeDesignMat(np.array(dataset)[:,0],np.array(dataset)[:,1],MAX_POWER_M,MAX_POWER_Y)

In [15]:
def testError(testX1,testX2,testX3,testY,w):
    testX = makeDesignMat(testX1,testX2,testX3,MAX_POWER_M,MAX_POWER_Y,MAX_POWER_MY)
    pred = np.dot(testX,w)
    err = 0.5 * (np.sum(np.square(pred-testY)))/len(testY)
    return err

In [19]:
def evaluate_moorepenrose(dataset):
    folds = cross_validation_split(dataset)
    err_list = []
    for fold in folds:
        trainDS = list(folds)
        trainDS.remove(fold)
        trainDS = np.array(trainDS)
        trainX1 = []
        trainX2 = []
        trainX3 = []
        trainY = []
        for i in range(len(trainDS)):
            trainX1.extend(trainDS[i][:,0])
            trainX2.extend(trainDS[i][:,1])
            trainX3.extend(trainDS[i][:,2])
            trainY.extend(trainDS[i][:,3])
        testDS = np.array(fold)
        testX1 = testDS[:,0]
        testX2 = testDS[:,1]
        testX3 = testDS[:,2]
        testY = testDS[:,3]
        X = makeDesignMat(trainX1,trainX2,trainX3,MAX_POWER_M,MAX_POWER_Y,MAX_POWER_MY)
        #w = np.dot(np.linalg.inv(np.dot(X.transpose(),X)),np.dot(X.transpose(),trainY))
        w = np.dot(np.linalg.inv(np.dot(X.transpose(),X) + REGULARIZATION_LAMBDA * np.identity(MAX_POWER_M+MAX_POWER_Y+MAX_POWER_MY+1) ),np.dot(X.transpose(),trainY))
        err = testError(testX1,testX2,testX3,testY,w)
        err_list.append(err)
    avg_error = np.mean(err_list)
    #print(err_list)
    return avg_error

In [32]:
MAX_POWER_M = 4
MAX_POWER_Y = 1
MAX_POWER_MY = 1
N_FOLDS = 10
REGULARIZATION_LAMBDA = 0.15

In [30]:
evaluate_moorepenrose(dataset)

4.611150177108769

In [33]:
leasterr = 99999
bestM,bestY,bestLambd = 0,0,0
for MAX_POWER_M in tqdm(range(1,10)):
    for MAX_POWER_Y in tqdm(range(1,10)):
        for MAX_POWER_MY in tqdm(range(1,10)):
            for REGULARIZATION_LAMBDA in [1e-12,1e-8,1e-4,1e-2,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]:
                sumerr = 0
                for i in range(5):
                    err = evaluate_moorepenrose(dataset)
                    sumerr+=err
                sumerr/=5
                if sumerr<leasterr:
                    leasterr = sumerr
                    bestM = MAX_POWER_M
                    bestY = MAX_POWER_Y
                    bestMY = MAX_POWER_MY
                    bestLambd = REGULARIZATION_LAMBDA

In [35]:
print(bestM,bestY,bestMY,bestLambd)

6 1 1 0.3


In [36]:
MAX_POWER_M = 6
MAX_POWER_Y = 1
MAX_POWER_MY = 1
N_FOLDS = 10
REGULARIZATION_LAMBDA = 0.3

In [55]:
evaluate_moorepenrose(dataset)

3.2640833694274916

In [59]:
testdf

,id
0,5/1/10
1,4/1/09
2,9/1/13
3,1/1/06
4,2/1/07
5,8/1/12
6,6/1/14
7,3/1/08
8,12/1/04
9,7/1/11


In [56]:
testdf = pd.read_csv('test.csv')

In [60]:
testdf[['month','date','year']] = testdf['id'].str.split('/',expand = True)

In [57]:
testdataset = []
for row in testdf.iterrows():
    testdataset.append([int(row[1]['month']),int(row[1]['year'])])

KeyError: 'month'

In [58]:
def trainModel(dataset):
    dataset = np.array(dataset)
    trainXX1 = dataset[:,0]
    trainXX2 = dataset[:,1]
    trainYY = dataset[:,2]
    finXX = makeDesignMat(trainXX1,trainXX2,MAX_POWER_M,MAX_POWER_Y)
    w = np.dot(np.linalg.inv(np.dot(finXX.transpose(),finXX) + REGULARIZATION_LAMBDA * np.identity(MAX_POWER_M+MAX_POWER_Y+1) ),np.dot(finXX.transpose(),trainYY))
    return w

In [25]:
def getPreds(testdataset):
    XX = np.array(testdataset)
    XX1 = XX[:,0]
    XX2 = XX[:,1]
    finX = makeDesignMat(XX1,XX2,MAX_POWER_M,MAX_POWER_Y)
    w = trainModel(dataset)
    pred = np.dot(finX,w)
    return pred

In [26]:
preds = getPreds(testdataset)

In [27]:
preds

array([25.91563975, 21.78508878,  4.82971767,  2.94834173,  7.57012947,
       12.59186129, 25.10910293, 14.73636348,  0.09550693, 20.46794847])

In [28]:
testdataset

[[5, 10],
 [4, 9],
 [9, 13],
 [1, 6],
 [2, 7],
 [8, 12],
 [6, 14],
 [3, 8],
 [12, 4],
 [7, 11]]

In [29]:
df[df['year']=='04']

,id,value,month,date,year
0,11/1/04,0.5992,11,1,04


In [30]:
newdf = pd.DataFrame({'id':testdf['id'],'value':preds})

In [31]:
newdf

,id,value
0,5/1/10,25.915640
1,4/1/09,21.785089
2,9/1/13,4.829718
3,1/1/06,2.948342
4,2/1/07,7.570129
5,8/1/12,12.591861
6,6/1/14,25.109103
7,3/1/08,14.736363
8,12/1/04,0.095507
9,7/1/11,20.467948


In [32]:
newdf.to_csv('out2.csv',index=False)